In [1]:
import pandas as pd
import re
from datetime import datetime

In [2]:
# 读入数据
data_batch = "test"
data_target = "rent"
path = f"ori_data//ruc_Class25Q2_{data_batch}_{data_target}.csv"
df = pd.read_csv(path)
print(f"数据维度：{df.shape}")

数据维度：(9773, 46)


In [3]:
# 处理“房屋户型”：“2室1厅1厨1卫”
def extract_room_shape(text):
    """
    参数:
    text: str - 户型字符串，如"2室1厅1厨1卫"
    
    返回:
    tuple - (室数, 厅数, 厨房数, 卫数)
    """
    # 初始化默认值
    if text == "":
        return -1,-1,-1,-1
    room = hall = kitchen = bathroom = 0
    
    # 使用正则表达式匹配数字和对应的单位
    pattern = r'(\d+)(室|厅|厨|卫)'
    matches = re.findall(pattern, text)
    
    # 将匹配结果分配到对应的变量
    for value, unit in matches:
        value = int(value)
        if unit == '室':
            room = value
        elif unit == '厅':
            hall = value
        elif unit == '厨':
            kitchen = value
        elif unit == '卫':
            bathroom = value
    
    return room, hall, kitchen, bathroom

In [4]:
# 检查缺失值
print(df['户型'].isnull().sum())
# 填充缺失值为空字符串
df['户型'] = df['户型'].fillna('')
# 应用提取函数并创建新列
df[['室数', '厅数', '厨房数', '卫数']] = df['户型'].apply(lambda x: pd.Series(extract_room_shape(x)))

print(df.columns)
# print(df.head())
print(f"数据维度：{df.shape}")

0
Index(['ID', '城市', '户型', '装修', '楼层', '面积', '朝向', '交易时间', '付款方式', '租赁方式', '电梯',
       '车位', '用水', '用电', '燃气', '采暖', '租期', '配套设施', 'lon', 'lat', '年份', '区县',
       '板块', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率',
       '容 积 率', '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费',
       '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数',
       '卫数'],
      dtype='object')
数据维度：(9773, 50)


In [5]:
df = df.drop("户型", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 49)


In [6]:
# 处理装修
def extract_fitment(text):
    """
    是否有精装修
    """
    if text == "精装修":
        return 1
    
    return 0

In [7]:
# 检查缺失值
target_column = "装修"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['精装修']] = df[target_column].apply(lambda x: pd.Series(extract_fitment(x)))

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

2682
Index(['ID', '城市', '装修', '楼层', '面积', '朝向', '交易时间', '付款方式', '租赁方式', '电梯', '车位',
       '用水', '用电', '燃气', '采暖', '租期', '配套设施', 'lon', 'lat', '年份', '区县', '板块',
       '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率',
       '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费',
       '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数',
       '精装修'],
      dtype='object')
        ID  城市   装修       楼层       面积   朝向        交易时间 付款方式 租赁方式 电梯  ...  \
0  2000000   1  精装修  低楼层/18层   86.94㎡  南 北  2025-08-01  NaN   整租  有  ...   
1  2000001  10  精装修   低楼层/8层   72.60㎡    南  2025-05-23  月付价   整租  有  ...   
2  2000002   3  精装修  中楼层/20层   98.00㎡    南  2025-02-18  NaN   整租  有  ...   
3  2000003   0  精装修  高楼层/12层   98.97㎡  东 西  2025-02-17  季付价   整租  有  ...   
4  2000004   3  精装修  中楼层/23层  170.53㎡    南  2025-03-24  季付价   整租  有  ...   

      停车位  停车费用     coord_x    coord_y                       客户反馈 室数 厅数  厨房数  \
0  1600.0   100  117.345687  40.44

In [8]:
df = df.drop("装修", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 49)


In [9]:
def extract_floor_info(text):
    """
    参数:
    text: str - 楼层字符串，如"中楼层/7层"、"低楼层/13层"、"30/45层"
    
    返回:
    tuple - (是否为底层, 是否为低层, 是否为中层, 是否为高层, 是否为顶层, 总楼层)
    """
    # 初始化默认值
    is_bottom = 0
    is_low = 0
    is_middle = 0
    is_high = 0
    is_top = 0
    total_floors = 0
    current_floor = 0
    
    # 提取总楼层数
    total_match = re.search(r'/(\d+)层', text)
    if total_match:
        total_floors = int(total_match.group(1))
    
    # 检查是否为"数字/数字层"格式（如"30/45层"）
    specific_floor_match = re.search(r'(\d+)/(\d+)层', text)
    
    if specific_floor_match:
        # 处理特殊情况：具体楼层/总楼层格式
        current_floor = int(specific_floor_match.group(1))
        total_floors = int(specific_floor_match.group(2))
        
        # 判断是否底层（1楼）
        if current_floor == 1:
            is_bottom = 1
        # 判断是否顶层（当前楼层等于总楼层）
        elif current_floor == total_floors:
            is_top = 1
        else:
            # 按三分之一划分楼层区域
            third = total_floors / 3
            
            if current_floor <= third:
                is_low = 1
            elif current_floor <= 2 * third:
                is_middle = 1
            else:
                is_high = 1
    else:
        # 原有逻辑：处理文字描述格式
        if '底层' in text:
            is_bottom = 1
        elif '低楼层' in text:
            is_low = 1
        elif '中楼层' in text:
            is_middle = 1
        elif '高楼层' in text:
            is_high = 1
        elif '顶层' in text:
            is_top = 1
    
    return is_bottom, is_low, is_middle, is_high, is_top, total_floors

In [10]:
# 检查缺失值
print(df['楼层'].isnull().sum())
# 填充缺失值为空字符串
df['楼层'] = df['楼层'].fillna('')
# 应用提取函数并创建新列
df[['是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层']] = df['楼层'].apply(
    lambda x: pd.Series(extract_floor_info(x))
)

# print(df.columns)
print(df.head(10))
print(f"数据维度：{df.shape}")

0
        ID  城市       楼层       面积   朝向        交易时间 付款方式 租赁方式 电梯    车位  ... 厅数  \
0  2000000   1  低楼层/18层   86.94㎡  南 北  2025-08-01  NaN   整租  有   NaN  ...  2   
1  2000001  10   低楼层/8层   72.60㎡    南  2025-05-23  月付价   整租  有   NaN  ...  1   
2  2000002   3  中楼层/20层   98.00㎡    南  2025-02-18  NaN   整租  有  租用车位  ...  2   
3  2000003   0  高楼层/12层   98.97㎡  东 西  2025-02-17  季付价   整租  有   NaN  ...  1   
4  2000004   3  中楼层/23层  170.53㎡    南  2025-03-24  季付价   整租  有  租用车位  ...  2   
5  2000005   2   30/45层   86.77㎡    南  2025-05-13  季付价   整租  有   NaN  ...  2   
6  2000006   9  中楼层/34层  135.00㎡  南 北  2025-04-14  NaN   整租  有   NaN  ...  2   
7  2000007   4   中楼层/6层   52.09㎡    南  2025-04-29  季付价   整租  无   NaN  ...  1   
8  2000008   9  高楼层/32层  106.50㎡    南  2025-09-20  NaN   整租  无   NaN  ...  1   
9  2000009   1   中楼层/9层   63.00㎡    南  2025-05-09  季付价   整租  有   NaN  ...  1   

  厨房数 卫数 精装修 是否为底层 是否为低层  是否为中层  是否为高层  是否为顶层  总楼层  
0   0  1   1     0     1      0      0      0   18  
1   0  1   

In [11]:
df = df.drop("楼层", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 54)


In [12]:
# 处理建筑面积
def extract_area_value(text):
    """    
    参数:
    text: str - 建筑面积字符串，如"$ 282.02m^{2} $"
    
    返回:
    float - 提取出的面积数值
    """
    match = re.search(r"(\d+\.?\d*)", text)
    
    if match:
        return float(match.group(1))
    else:
        return 0  # 如果没有匹配到数字，返回0

In [13]:
df['建筑面积值'] = df['面积'].apply(extract_area_value)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

Index(['ID', '城市', '面积', '朝向', '交易时间', '付款方式', '租赁方式', '电梯', '车位', '用水', '用电',
       '燃气', '采暖', '租期', '配套设施', 'lon', 'lat', '年份', '区县', '板块', '环线位置',
       '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率',
       '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费',
       '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数',
       '精装修', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值'],
      dtype='object')
        ID  城市       面积   朝向        交易时间 付款方式 租赁方式 电梯    车位   用水  ... 厨房数 卫数  \
0  2000000   1   86.94㎡  南 北  2025-08-01  NaN   整租  有   NaN   民水  ...   0  1   
1  2000001  10   72.60㎡    南  2025-05-23  月付价   整租  有   NaN   民水  ...   0  1   
2  2000002   3   98.00㎡    南  2025-02-18  NaN   整租  有  租用车位  NaN  ...   0  1   
3  2000003   0   98.97㎡  东 西  2025-02-17  季付价   整租  有   NaN   民水  ...   0  1   
4  2000004   3  170.53㎡    南  2025-03-24  季付价   整租  有  租用车位   民水  ...   0  2   

  精装修 是否为底层 是否为低层  是否为中层  是否为高层  是否为顶层  总楼层   

In [14]:
df = df.drop("面积", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 54)


In [15]:
# 处理房屋朝向
def extract_room_direction(text):
    south = north = west = east = 0
    
    if "南" in text:
        south = 1
    if "北" in text:
        north = 1
    if "西" in text:
        west = 1
    if "东" in text:
        east = 1
    
    return south, north, west, east

In [16]:
# 检查缺失值
print(df['朝向'].isnull().sum())
# 填充缺失值为空字符串
df['朝向'] = df['朝向'].fillna('')
# 应用提取函数并创建新列
df[['向南', '向北', '向西', '向东']] = df['朝向'].apply(
    lambda x: pd.Series(extract_room_direction(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

0
Index(['ID', '城市', '朝向', '交易时间', '付款方式', '租赁方式', '电梯', '车位', '用水', '用电', '燃气',
       '采暖', '租期', '配套设施', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别',
       '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费',
       '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用',
       'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '精装修', '是否为底层',
       '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '向南', '向北', '向西',
       '向东'],
      dtype='object')
        ID  城市   朝向        交易时间 付款方式 租赁方式 电梯    车位   用水   用电  ... 是否为低层 是否为中层  \
0  2000000   1  南 北  2025-08-01  NaN   整租  有   NaN   民水   民电  ...     1     0   
1  2000001  10    南  2025-05-23  月付价   整租  有   NaN   民水   民电  ...     1     0   
2  2000002   3    南  2025-02-18  NaN   整租  有  租用车位  NaN  NaN  ...     0     1   
3  2000003   0  东 西  2025-02-17  季付价   整租  有   NaN   民水   民电  ...     0     0   
4  2000004   3    南  2025-03-24  季付价   整租  有  租用车位   民水   民电  ...     0     1   

  是否为高层 是否为顶层

In [17]:
df = df.drop("朝向", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 57)


In [18]:
# 检查缺失值
target_column = "交易时间"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')

# 将"交易时间"列转换为datetime类型
df['交易时间dt'] = pd.to_datetime(df[target_column])
# 设置参考日期为2025年12月31日
reference_date = datetime(2025, 12, 31)
# 计算每个交易日期距离参考日期的天数
df['交易距今'] = (reference_date - df['交易时间dt']).dt.days

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

0
Index(['ID', '城市', '交易时间', '付款方式', '租赁方式', '电梯', '车位', '用水', '用电', '燃气', '采暖',
       '租期', '配套设施', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '建筑年代',
       '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构',
       '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用',
       'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '精装修', '是否为底层',
       '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '向南', '向北', '向西',
       '向东', '交易时间dt', '交易距今'],
      dtype='object')
        ID  城市        交易时间 付款方式 租赁方式 电梯    车位   用水   用电 燃气  ... 是否为高层 是否为顶层  \
0  2000000   1  2025-08-01  NaN   整租  有   NaN   民水   民电  有  ...     0     0   
1  2000001  10  2025-05-23  月付价   整租  有   NaN   民水   民电  有  ...     0     0   
2  2000002   3  2025-02-18  NaN   整租  有  租用车位  NaN  NaN  有  ...     0     0   
3  2000003   0  2025-02-17  季付价   整租  有   NaN   民水   民电  有  ...     1     0   
4  2000004   3  2025-03-24  季付价   整租  有  租用车位   民水   民电  有  ...     0     0   

  总楼层   建筑面积值

In [19]:
df = df.drop("交易时间", axis=1)
df = df.drop("交易时间dt", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 57)


In [20]:
# 处理付款方式
def extract_pay_way(text):
    jifu = shuangyuefu = yuefu = bannianfu = nianfu = 0
    
    if "季付价" in text:
        jifu = 1
    if "双月付价" in text:
        shuangyuefu = 1
    elif "月付价" in text:
        yuefu = 1
    if "半年付价" in text:
        bannianfu = 1
    elif "年付价" in text:
        nianfu = 1
    
    return jifu, shuangyuefu, yuefu, bannianfu, nianfu

In [21]:
# 检查缺失值
print(df['付款方式'].isnull().sum())
# 填充缺失值为空字符串
df['付款方式'] = df['付款方式'].fillna('')
# 应用提取函数并创建新列
df[['季付', '双月付', '月付', '半年付价', '年付']] = df['付款方式'].apply(
    lambda x: pd.Series(extract_pay_way(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

2387
Index(['ID', '城市', '付款方式', '租赁方式', '电梯', '车位', '用水', '用电', '燃气', '采暖', '租期',
       '配套设施', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '建筑年代', '开发商',
       '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构', '物业办公电话',
       '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x',
       'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '精装修', '是否为底层', '是否为低层',
       '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '向南', '向北', '向西', '向东',
       '交易距今', '季付', '双月付', '月付', '半年付价', '年付'],
      dtype='object')
        ID  城市 付款方式 租赁方式 电梯    车位   用水   用电 燃气    采暖  ... 向南 向北  向西  向东  交易距今  \
0  2000000   1        整租  有   NaN   民水   民电  有   自采暖  ...  1  1   0   0   152   
1  2000001  10  月付价   整租  有   NaN   民水   民电  有   NaN  ...  1  0   0   0   222   
2  2000002   3        整租  有  租用车位  NaN  NaN  有   NaN  ...  1  0   0   0   316   
3  2000003   0  季付价   整租  有   NaN   民水   民电  有  集中供暖  ...  0  0   1   1   317   
4  2000004   3  季付价   整租  有  租用车位   民水   民电  有   NaN  ...  1  0  

In [22]:
df = df.drop("付款方式", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 61)


In [23]:
# 处理租赁方式
def extract_rent_way(text):
    zhengzu = hezu = 0
    
    if "整租" in text:
        zhengzu = 1
    if "合租" in text:
        hezu = 1
    
    return zhengzu, hezu

In [24]:
# 检查缺失值
print(df['租赁方式'].isnull().sum())
# 填充缺失值为空字符串
df['租赁方式'] = df['租赁方式'].fillna('')
# 应用提取函数并创建新列
df[['整租', '合租']] = df['租赁方式'].apply(
    lambda x: pd.Series(extract_rent_way(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

0
Index(['ID', '城市', '租赁方式', '电梯', '车位', '用水', '用电', '燃气', '采暖', '租期', '配套设施',
       'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数',
       '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构', '物业办公电话', '产权描述',
       '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y',
       '客户反馈', '室数', '厅数', '厨房数', '卫数', '精装修', '是否为底层', '是否为低层', '是否为中层',
       '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '向南', '向北', '向西', '向东', '交易距今', '季付',
       '双月付', '月付', '半年付价', '年付', '整租', '合租'],
      dtype='object')
        ID  城市 租赁方式 电梯    车位   用水   用电 燃气    采暖    租期  ... 向西  向东  交易距今  季付  \
0  2000000   1   整租  有   NaN   民水   民电  有   自采暖   NaN  ...  0   0   152   0   
1  2000001  10   整租  有   NaN   民水   民电  有   NaN   NaN  ...  0   0   222   0   
2  2000002   3   整租  有  租用车位  NaN  NaN  有   NaN  1年以内  ...  0   0   316   0   
3  2000003   0   整租  有   NaN   民水   民电  有  集中供暖   NaN  ...  1   1   317   1   
4  2000004   3   整租  有  租用车位   民水   民电  有   NaN   NaN  ...  0   0   282   

In [25]:
df = df.drop("租赁方式", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 62)


In [26]:
# 处理电梯
def extract_ele(text):
    is_elev = -1
    
    if "有" in text:
        is_elev = 1
    if "无" in text:
        is_elev = 0
    
    return is_elev

In [27]:
# 检查缺失值
print(df['电梯'].isnull().sum())
# 填充缺失值为空字符串
df['电梯'] = df['电梯'].fillna('')
# 应用提取函数并创建新列
df[['是否有电梯']] = df['电梯'].apply(
    lambda x: pd.Series(extract_ele(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

0
Index(['ID', '城市', '电梯', '车位', '用水', '用电', '燃气', '采暖', '租期', '配套设施', 'lon',
       'lat', '年份', '区县', '板块', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数',
       '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖',
       '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数',
       '厅数', '厨房数', '卫数', '精装修', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层',
       '总楼层', '建筑面积值', '向南', '向北', '向西', '向东', '交易距今', '季付', '双月付', '月付',
       '半年付价', '年付', '整租', '合租', '是否有电梯'],
      dtype='object')
        ID  城市 电梯    车位   用水   用电 燃气    采暖    租期  \
0  2000000   1  有   NaN   民水   民电  有   自采暖   NaN   
1  2000001  10  有   NaN   民水   民电  有   NaN   NaN   
2  2000002   3  有  租用车位  NaN  NaN  有   NaN  1年以内   
3  2000003   0  有   NaN   民水   民电  有  集中供暖   NaN   
4  2000004   3  有  租用车位   民水   民电  有   NaN   NaN   

                             配套设施  ...  向东  交易距今  季付  双月付  月付 半年付价 年付 整租 合租  \
0  洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、宽带、天然  ...   0   152   0    0   0    0  0  1  0 

In [28]:
df = df.drop("电梯", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 62)


In [29]:
# 处理车位
def extract_park_where(text):
    zuyong = mianfei = 0
    
    if "租用车位" in text:
        zuyong = 1
    if "免费使用" in text:
        mianfei = 1
    
    return zuyong, mianfei

In [30]:
# 检查缺失值
print(df['车位'].isnull().sum())
# 填充缺失值为空字符串
df['车位'] = df['车位'].fillna('')
# 应用提取函数并创建新列
df[['租用车位', '免费使用']] = df['车位'].apply(
    lambda x: pd.Series(extract_park_where(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

7611
Index(['ID', '城市', '车位', '用水', '用电', '燃气', '采暖', '租期', '配套设施', 'lon', 'lat',
       '年份', '区县', '板块', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司',
       '绿 化 率', '容 积 率', '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电',
       '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数',
       '厨房数', '卫数', '精装修', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层',
       '建筑面积值', '向南', '向北', '向西', '向东', '交易距今', '季付', '双月付', '月付', '半年付价',
       '年付', '整租', '合租', '是否有电梯', '租用车位', '免费使用'],
      dtype='object')
        ID  城市    车位   用水   用电 燃气    采暖    租期                            配套设施  \
0  2000000   1         民水   民电  有   自采暖   NaN  洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、宽带、天然   
1  2000001  10         民水   民电  有   NaN   NaN              洗衣机、空调、衣柜、冰箱、热水器、床   
2  2000002   3  租用车位  NaN  NaN  有   NaN  1年以内       洗衣机、空调、衣柜、电视、冰箱、热水器、床、天然气   
3  2000003   0         民水   民电  有  集中供暖   NaN                             NaN   
4  2000004   3  租用车位   民水   民电  有   NaN   NaN     

In [31]:
df = df.drop("车位", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 63)


In [32]:
# 处理供水
def extract_water_giving(text):
    minshui = shangshui = 0
    if "民水" in text:
        minshui = 1
    if "商水" in text:
        shangshui = 1
    
    return minshui, shangshui

In [33]:
# 检查缺失值
target_column = "用水"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['民水供水1', '商水供水1']] = df[target_column].apply(
    lambda x: pd.Series(extract_water_giving(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

2273
Index(['ID', '城市', '用水', '用电', '燃气', '采暖', '租期', '配套设施', 'lon', 'lat', '年份',
       '区县', '板块', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司',
       '绿 化 率', '容 积 率', '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电',
       '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数',
       '厨房数', '卫数', '精装修', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层',
       '建筑面积值', '向南', '向北', '向西', '向东', '交易距今', '季付', '双月付', '月付', '半年付价',
       '年付', '整租', '合租', '是否有电梯', '租用车位', '免费使用', '民水供水1', '商水供水1'],
      dtype='object')
        ID  城市  用水   用电 燃气    采暖    租期                            配套设施  \
0  2000000   1  民水   民电  有   自采暖   NaN  洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、宽带、天然   
1  2000001  10  民水   民电  有   NaN   NaN              洗衣机、空调、衣柜、冰箱、热水器、床   
2  2000002   3      NaN  有   NaN  1年以内       洗衣机、空调、衣柜、电视、冰箱、热水器、床、天然气   
3  2000003   0  民水   民电  有  集中供暖   NaN                             NaN   
4  2000004   3  民水   民电  有   NaN   NaN       洗衣机、空调、衣柜、电视、冰箱、热水器、床、天然气   

In [34]:
df = df.drop("用水", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 64)


In [35]:
# 处理供电
def extract_elec_giving(text):
    mindian = shangdian = 0
    if "民电" in text:
        mindian = 1
    if "商电" in text:
        shangdian = 1
    
    return mindian, shangdian

In [36]:
# 检查缺失值
target_column = "用电"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['民电供电1', '商电供电1']] = df[target_column].apply(
    lambda x: pd.Series(extract_water_giving(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

2227
Index(['ID', '城市', '用电', '燃气', '采暖', '租期', '配套设施', 'lon', 'lat', '年份', '区县',
       '板块', '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率',
       '容 积 率', '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费',
       '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数',
       '卫数', '精装修', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层',
       '建筑面积值', '向南', '向北', '向西', '向东', '交易距今', '季付', '双月付', '月付', '半年付价',
       '年付', '整租', '合租', '是否有电梯', '租用车位', '免费使用', '民水供水1', '商水供水1', '民电供电1',
       '商电供电1'],
      dtype='object')
        ID  城市  用电 燃气    采暖    租期                            配套设施         lon  \
0  2000000   1  民电  有   自采暖   NaN  洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、宽带、天然  117.345687   
1  2000001  10  民电  有   NaN   NaN              洗衣机、空调、衣柜、冰箱、热水器、床  114.279469   
2  2000002   3      有   NaN  1年以内       洗衣机、空调、衣柜、电视、冰箱、热水器、床、天然气  121.684578   
3  2000003   0  民电  有  集中供暖   NaN                             NaN  117.323901   
4  2000004   3  民电 

In [37]:
df = df.drop("用电", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 65)


In [38]:
# 处理燃气
def extract_burn(text):
    is_elev = -1
    
    if "有" in text:
        is_elev = 1
    if "无" in text:
        is_elev = 0
    
    return is_elev

In [39]:
# 检查缺失值
print(df['燃气'].isnull().sum())
# 填充缺失值为空字符串
df['燃气'] = df['燃气'].fillna('')
# 应用提取函数并创建新列
df[['是否有燃气']] = df['燃气'].apply(
    lambda x: pd.Series(extract_burn(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

591
Index(['ID', '城市', '燃气', '采暖', '租期', '配套设施', 'lon', 'lat', '年份', '区县', '板块',
       '环线位置', '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率',
       '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费',
       '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数',
       '精装修', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值',
       '向南', '向北', '向西', '向东', '交易距今', '季付', '双月付', '月付', '半年付价', '年付', '整租',
       '合租', '是否有电梯', '租用车位', '免费使用', '民水供水1', '商水供水1', '民电供电1', '商电供电1',
       '是否有燃气'],
      dtype='object')
        ID  城市 燃气    采暖    租期                            配套设施         lon  \
0  2000000   1  有   自采暖   NaN  洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、宽带、天然  117.345687   
1  2000001  10  有   NaN   NaN              洗衣机、空调、衣柜、冰箱、热水器、床  114.279469   
2  2000002   3  有   NaN  1年以内       洗衣机、空调、衣柜、电视、冰箱、热水器、床、天然气  121.684578   
3  2000003   0  有  集中供暖   NaN                             NaN  117.323901   
4  2000004   3  有   NaN   NaN       洗

In [40]:
df = df.drop("燃气", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 65)


In [41]:
# 处理供暖
def extract_warm_giving(text):
    jizhong = zi = 0
    if "集中供暖" in text:
        jizhong = 1
    if "自采暖" in text:
        zi = 1
    
    return jizhong, zi

In [42]:
# 检查缺失值
target_column = "采暖"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['集中供暖1', '自采暖1']] = df[target_column].apply(
    lambda x: pd.Series(extract_warm_giving(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

6560
Index(['ID', '城市', '采暖', '租期', '配套设施', 'lon', 'lat', '年份', '区县', '板块', '环线位置',
       '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率',
       '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费',
       '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数',
       '精装修', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值',
       '向南', '向北', '向西', '向东', '交易距今', '季付', '双月付', '月付', '半年付价', '年付', '整租',
       '合租', '是否有电梯', '租用车位', '免费使用', '民水供水1', '商水供水1', '民电供电1', '商电供电1',
       '是否有燃气', '集中供暖1', '自采暖1'],
      dtype='object')
        ID  城市    采暖    租期                            配套设施         lon  \
0  2000000   1   自采暖   NaN  洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、宽带、天然  117.345687   
1  2000001  10         NaN              洗衣机、空调、衣柜、冰箱、热水器、床  114.279469   
2  2000002   3        1年以内       洗衣机、空调、衣柜、电视、冰箱、热水器、床、天然气  121.684578   
3  2000003   0  集中供暖   NaN                             NaN  117.323901   
4  2000004   3         NaN       洗衣机、空调、

In [43]:
df = df.drop("采暖", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 66)


In [44]:
def calculate_lease_months(time_desc):
    """
    从时间描述计算租赁月份数
    
    参数:
    time_desc: str - 时间描述字符串
    
    返回:
    float - 计算的月份数
    """
    # 检查是否为空值
    if pd.isna(time_desc):
        return -1.0
    
    # 转换为字符串并去除前后空格
    desc = str(time_desc).strip()
    
    try:
        # 处理"以内"情况 - 时间乘以0.75
        if '以内' in desc:
            # 提取数字
            match = re.search(r'(\d+(?:\.\d+)?)', desc)
            if match:
                num = float(match.group(1))
                # 检查单位是年还是月
                if '年' in desc:
                    return num * 12 * 0.75
                else:  # 默认是月
                    return num * 0.75
        
        # 处理"以上"情况 - 时间乘以1.5
        elif '以上' in desc:
            # 提取数字
            match = re.search(r'(\d+(?:\.\d+)?)', desc)
            if match:
                num = float(match.group(1))
                # 检查单位是年还是月
                if '年' in desc:
                    return num * 12 * 1.5
                else:  # 默认是月
                    return num * 1.5
        
        # 处理范围情况"~" - 取平均值
        elif '~' in desc:
            # 提取两个数字
            matches = re.findall(r'(\d+(?:\.\d+)?)', desc)
            if len(matches) == 2:
                num1 = float(matches[0])
                num2 = float(matches[1])
                avg = (num1 + num2) / 2
                
                # 检查单位是年还是月
                if '年' in desc:
                    return avg * 12
                else:  # 默认是月
                    return avg
        
        # 处理单个数字情况
        else:
            # 提取数字
            match = re.search(r'(\d+(?:\.\d+)?)', desc)
            if match:
                num = float(match.group(1))
                # 检查单位是年还是月
                if '年' in desc:
                    return num * 12
                else:  # 默认是月
                    return num
        
        # 如果以上都不匹配，返回-1
        return -1.0
    
    except (ValueError, IndexError, AttributeError):
        return -1.0

In [45]:
# 检查缺失值
target_column = "租期"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['租赁时间']] = df[target_column].apply(
    lambda x: pd.Series(calculate_lease_months(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

5175
Index(['ID', '城市', '租期', '配套设施', 'lon', 'lat', '年份', '区县', '板块', '环线位置',
       '物业类别', '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率',
       '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费',
       '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数',
       '精装修', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值',
       '向南', '向北', '向西', '向东', '交易距今', '季付', '双月付', '月付', '半年付价', '年付', '整租',
       '合租', '是否有电梯', '租用车位', '免费使用', '民水供水1', '商水供水1', '民电供电1', '商电供电1',
       '是否有燃气', '集中供暖1', '自采暖1', '租赁时间'],
      dtype='object')
        ID  城市    租期                            配套设施         lon        lat  \
0  2000000   1        洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、宽带、天然  117.345687  40.447235   
1  2000001  10                    洗衣机、空调、衣柜、冰箱、热水器、床  114.279469  24.158183   
2  2000002   3  1年以内       洗衣机、空调、衣柜、电视、冰箱、热水器、床、天然气  121.684578  32.198660   
3  2000003   0                                   NaN  117.323901  40.935338   
4  2000004   

In [46]:
df = df.drop("租期", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 66)


In [47]:
# 处理配套设施
def extract_equipment(text):
    if text == "":
        return 0
    else:
        return len(text.split("、"))

In [48]:
# 检查缺失值
target_column = "配套设施"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['设施情况']] = df[target_column].apply(
    lambda x: pd.Series(extract_equipment(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

3694
Index(['ID', '城市', '配套设施', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别',
       '建筑年代', '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费',
       '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用',
       'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '精装修', '是否为底层',
       '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '向南', '向北', '向西',
       '向东', '交易距今', '季付', '双月付', '月付', '半年付价', '年付', '整租', '合租', '是否有电梯',
       '租用车位', '免费使用', '民水供水1', '商水供水1', '民电供电1', '商电供电1', '是否有燃气', '集中供暖1',
       '自采暖1', '租赁时间', '设施情况'],
      dtype='object')
        ID  城市                            配套设施         lon        lat      年份  \
0  2000000   1  洗衣机、空调、衣柜、电视、冰箱、热水器、床、暖气、宽带、天然  117.345687  40.447235  2023.0   
1  2000001  10              洗衣机、空调、衣柜、冰箱、热水器、床  114.279469  24.158183  2022.0   
2  2000002   3       洗衣机、空调、衣柜、电视、冰箱、热水器、床、天然气  121.684578  32.198660  2022.0   
3  2000003   0                                  117.323901  40.935338  2022.0   
4

In [49]:
df = df.drop("配套设施", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 66)


In [50]:
def extract_building_year(building_year):
    """
    参数:
    building_year: str - 建筑年代字符串
    
    返回:
    float - 处理后的年份值
    """    
    # 转换为字符串类型
    building_year = str(building_year)
    
    # 2002-2006年
    if '-' in building_year:
        # 使用正则表达式提取两个年份
        years = re.findall(r'\d+', building_year)
        start_year = int(years[0])
        end_year = int(years[1])
        return (start_year + end_year) / 2
    
    # 2004年
    elif '年' in building_year:
        # 提取年份数字
        year_match = re.search(r'\d+', building_year)
        return float(year_match.group())

In [51]:
# 检查缺失值
target_column = "建筑年代"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[["房屋年份"]] = df[target_column].apply(
    lambda x: pd.Series(extract_building_year(x), dtype='object')
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

3381
Index(['ID', '城市', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '建筑年代',
       '开发商', '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构',
       '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用',
       'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '精装修', '是否为底层',
       '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '向南', '向北', '向西',
       '向东', '交易距今', '季付', '双月付', '月付', '半年付价', '年付', '整租', '合租', '是否有电梯',
       '租用车位', '免费使用', '民水供水1', '商水供水1', '民电供电1', '商电供电1', '是否有燃气', '集中供暖1',
       '自采暖1', '租赁时间', '设施情况', '房屋年份'],
      dtype='object')
        ID  城市         lon        lat      年份     区县     板块  环线位置  \
0  2000000   1  117.345687  40.447235  2023.0   24.0  303.0   NaN   
1  2000001  10  114.279469  24.158183  2022.0  105.0  985.0   NaN   
2  2000002   3  121.684578  32.198660  2022.0   21.0  387.0   NaN   
3  2000003   0  117.323901  40.935338  2022.0   81.0  111.0  三至四环   
4  2000004   3  121.718296  32.335562  2022.0   49.0  732.0  

In [52]:
df = df.drop("建筑年代", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 66)


In [53]:
# 检查缺失值
target_column = "房屋总数"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('-1户')
# 使用正则表达式提取数字
df['总户数'] = df[target_column].str.extract(r'(\d+)').astype(int)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

923
Index(['ID', '城市', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '开发商',
       '房屋总数', '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构', '物业办公电话',
       '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x',
       'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '精装修', '是否为底层', '是否为低层',
       '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '向南', '向北', '向西', '向东',
       '交易距今', '季付', '双月付', '月付', '半年付价', '年付', '整租', '合租', '是否有电梯', '租用车位',
       '免费使用', '民水供水1', '商水供水1', '民电供电1', '商电供电1', '是否有燃气', '集中供暖1', '自采暖1',
       '租赁时间', '设施情况', '房屋年份', '总户数'],
      dtype='object')
        ID  城市         lon        lat      年份     区县     板块  环线位置  \
0  2000000   1  117.345687  40.447235  2023.0   24.0  303.0   NaN   
1  2000001  10  114.279469  24.158183  2022.0  105.0  985.0   NaN   
2  2000002   3  121.684578  32.198660  2022.0   21.0  387.0   NaN   
3  2000003   0  117.323901  40.935338  2022.0   81.0  111.0  三至四环   
4  2000004   3  121.718296  32.335562  2022.0   49.0  732.0   N

In [54]:
df = df.drop("房屋总数", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 66)


In [55]:
# 检查缺失值
target_column = "楼栋总数"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('-1栋')
# 使用正则表达式提取数字
df['总楼数'] = df[target_column].str.extract(r'(\d+)').astype(int)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

923
Index(['ID', '城市', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '开发商',
       '楼栋总数', '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构', '物业办公电话', '产权描述',
       '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y',
       '客户反馈', '室数', '厅数', '厨房数', '卫数', '精装修', '是否为底层', '是否为低层', '是否为中层',
       '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '向南', '向北', '向西', '向东', '交易距今', '季付',
       '双月付', '月付', '半年付价', '年付', '整租', '合租', '是否有电梯', '租用车位', '免费使用', '民水供水1',
       '商水供水1', '民电供电1', '商电供电1', '是否有燃气', '集中供暖1', '自采暖1', '租赁时间', '设施情况',
       '房屋年份', '总户数', '总楼数'],
      dtype='object')
        ID  城市         lon        lat      年份     区县     板块  环线位置  \
0  2000000   1  117.345687  40.447235  2023.0   24.0  303.0   NaN   
1  2000001  10  114.279469  24.158183  2022.0  105.0  985.0   NaN   
2  2000002   3  121.684578  32.198660  2022.0   21.0  387.0   NaN   
3  2000003   0  117.323901  40.935338  2022.0   81.0  111.0  三至四环   
4  2000004   3  121.718296  32.335562  2022.0   49.0  732.0   Na

In [56]:
df = df.drop("楼栋总数", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 66)


In [57]:
# 检查缺失值
target_column = "绿 化 率"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('0.0')
# 提取数字
df['绿化率'] = df[target_column].str.rstrip('%').astype(float)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

3019
Index(['ID', '城市', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '开发商',
       '物业公司', '绿 化 率', '容 积 率', '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖',
       '供电', '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数',
       '厅数', '厨房数', '卫数', '精装修', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层',
       '总楼层', '建筑面积值', '向南', '向北', '向西', '向东', '交易距今', '季付', '双月付', '月付',
       '半年付价', '年付', '整租', '合租', '是否有电梯', '租用车位', '免费使用', '民水供水1', '商水供水1',
       '民电供电1', '商电供电1', '是否有燃气', '集中供暖1', '自采暖1', '租赁时间', '设施情况', '房屋年份',
       '总户数', '总楼数', '绿化率'],
      dtype='object')
        ID  城市         lon        lat      年份     区县     板块  环线位置  \
0  2000000   1  117.345687  40.447235  2023.0   24.0  303.0   NaN   
1  2000001  10  114.279469  24.158183  2022.0  105.0  985.0   NaN   
2  2000002   3  121.684578  32.198660  2022.0   21.0  387.0   NaN   
3  2000003   0  117.323901  40.935338  2022.0   81.0  111.0  三至四环   
4  2000004   3  121.718296  32.335562  2022.0   49.0  732.0   Na

In [58]:
df = df.drop("绿 化 率", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 66)


In [59]:
# 检查缺失值
target_column = "容 积 率"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('-1')
# 提取数字
df['容积率'] = df[target_column].astype(float)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

2988
Index(['ID', '城市', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '开发商',
       '物业公司', '容 积 率', '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电',
       '燃气费', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数',
       '厨房数', '卫数', '精装修', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层',
       '建筑面积值', '向南', '向北', '向西', '向东', '交易距今', '季付', '双月付', '月付', '半年付价',
       '年付', '整租', '合租', '是否有电梯', '租用车位', '免费使用', '民水供水1', '商水供水1', '民电供电1',
       '商电供电1', '是否有燃气', '集中供暖1', '自采暖1', '租赁时间', '设施情况', '房屋年份', '总户数', '总楼数',
       '绿化率', '容积率'],
      dtype='object')
        ID  城市         lon        lat      年份     区县     板块  环线位置  \
0  2000000   1  117.345687  40.447235  2023.0   24.0  303.0   NaN   
1  2000001  10  114.279469  24.158183  2022.0  105.0  985.0   NaN   
2  2000002   3  121.684578  32.198660  2022.0   21.0  387.0   NaN   
3  2000003   0  117.323901  40.935338  2022.0   81.0  111.0  三至四环   
4  2000004   3  121.718296  32.335562  2022.0   49.0  732.0   NaN 

In [60]:
df = df.drop("容 积 率", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 66)


In [61]:
def extract_property_fee(value):
    """
    参数:
    value: str - 物业费字符串
    
    返回:
    float - 提取的物业费数值，或-1（如果无法提取）
    """
    # 检查是否为空值
    if value == "":
        return -1
    # 转换为字符串并去除前后空格
    value_str = str(value)
    # 检查是否以"/月/m²"结尾
    if not value_str.endswith("元/月/㎡"):
        return -1
    
    try:
        # 提取数字部分（可能包含范围符号"-"）
        # 使用正则表达式匹配数字和可能的范围
        pattern = r'([\d.-]+)元/月/㎡'
        match = re.search(pattern, value_str)
        
        if match:
            num_part = match.group(1)
            
            # 检查是否是范围（包含"-"）
            if '-' in num_part:
                # 处理范围，取平均值
                parts = num_part.split('-')
                if len(parts) == 2:
                    start = float(parts[0])
                    end = float(parts[1])
                    return (start + end) / 2
                else:
                    return -1.0
            else:
                # 处理单个数值
                return float(num_part)
        else:
            return -1.0
    except (ValueError, AttributeError):
        return -1.0

In [62]:
# 检查缺失值
target_column = "物 业 费"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 使用正则表达式提取数字
df[["物业费"]] = df[target_column].apply(
    lambda x: pd.Series(extract_property_fee(x), dtype='object')
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

2804
Index(['ID', '城市', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '开发商',
       '物业公司', '物 业 费', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费',
       '供热费', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数',
       '卫数', '精装修', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层',
       '建筑面积值', '向南', '向北', '向西', '向东', '交易距今', '季付', '双月付', '月付', '半年付价',
       '年付', '整租', '合租', '是否有电梯', '租用车位', '免费使用', '民水供水1', '商水供水1', '民电供电1',
       '商电供电1', '是否有燃气', '集中供暖1', '自采暖1', '租赁时间', '设施情况', '房屋年份', '总户数', '总楼数',
       '绿化率', '容积率', '物业费'],
      dtype='object')
        ID  城市         lon        lat      年份     区县     板块  环线位置  \
0  2000000   1  117.345687  40.447235  2023.0   24.0  303.0   NaN   
1  2000001  10  114.279469  24.158183  2022.0  105.0  985.0   NaN   
2  2000002   3  121.684578  32.198660  2022.0   21.0  387.0   NaN   
3  2000003   0  117.323901  40.935338  2022.0   81.0  111.0  三至四环   
4  2000004   3  121.718296  32.335562  2022.0   49.0  732.0   NaN   

In [63]:
df = df.drop("物 业 费", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 66)


In [64]:
# 处理建筑结构
def extract_structure(text):
    talou = banlou = pingfang = 0
    if "塔楼" in text:
        talou = 1
    if "板楼" in text:
        banlou = 1
    if "平房" in text:
        pingfang = 1
    if "塔板结合" in text:
        talou = 1
        banlou = 1
        
    return talou, banlou, pingfang

In [65]:
# 检查缺失值
target_column = "建筑结构"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['塔楼', '板楼', '平房']] = df[target_column].apply(
    lambda x: pd.Series(extract_structure(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

2699
Index(['ID', '城市', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '开发商',
       '物业公司', '建筑结构', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位',
       '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '精装修',
       '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '向南', '向北',
       '向西', '向东', '交易距今', '季付', '双月付', '月付', '半年付价', '年付', '整租', '合租',
       '是否有电梯', '租用车位', '免费使用', '民水供水1', '商水供水1', '民电供电1', '商电供电1', '是否有燃气',
       '集中供暖1', '自采暖1', '租赁时间', '设施情况', '房屋年份', '总户数', '总楼数', '绿化率', '容积率',
       '物业费', '塔楼', '板楼', '平房'],
      dtype='object')
        ID  城市         lon        lat      年份     区县     板块  环线位置  \
0  2000000   1  117.345687  40.447235  2023.0   24.0  303.0   NaN   
1  2000001  10  114.279469  24.158183  2022.0  105.0  985.0   NaN   
2  2000002   3  121.684578  32.198660  2022.0   21.0  387.0   NaN   
3  2000003   0  117.323901  40.935338  2022.0   81.0  111.0  三至四环   
4  2000004   3  121.718296  32.335562  2022.0   49.0  732.0

In [66]:
df = df.drop("建筑结构", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 68)


In [67]:
# 处理供水
def extract_water_giving(text):
    minshui = shangshui = 0
    if "民水" in text:
        minshui = 1
    if "商水" in text:
        shangshui = 1
    
    return minshui, shangshui

In [68]:
# 检查缺失值
target_column = "供水"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['民水供水2', '商水供水2']] = df[target_column].apply(
    lambda x: pd.Series(extract_water_giving(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

2762
Index(['ID', '城市', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '开发商',
       '物业公司', '物业办公电话', '产权描述', '供水', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用',
       'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '精装修', '是否为底层',
       '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '向南', '向北', '向西',
       '向东', '交易距今', '季付', '双月付', '月付', '半年付价', '年付', '整租', '合租', '是否有电梯',
       '租用车位', '免费使用', '民水供水1', '商水供水1', '民电供电1', '商电供电1', '是否有燃气', '集中供暖1',
       '自采暖1', '租赁时间', '设施情况', '房屋年份', '总户数', '总楼数', '绿化率', '容积率', '物业费', '塔楼',
       '板楼', '平房', '民水供水2', '商水供水2'],
      dtype='object')
        ID  城市         lon        lat      年份     区县     板块  环线位置  \
0  2000000   1  117.345687  40.447235  2023.0   24.0  303.0   NaN   
1  2000001  10  114.279469  24.158183  2022.0  105.0  985.0   NaN   
2  2000002   3  121.684578  32.198660  2022.0   21.0  387.0   NaN   
3  2000003   0  117.323901  40.935338  2022.0   81.0  111.0  三至四环   
4  2000004   3  121.718296  32.335562  2022.0   4

In [69]:
df['民水供水'] = df.apply(lambda row: max(row["民水供水1"], row["民水供水2"]), axis=1)
df['商水供水'] = df.apply(lambda row: max(row["商水供水1"], row["商水供水2"]), axis=1)

In [70]:
df = df.drop("民水供水1", axis=1)
df = df.drop("民水供水2", axis=1)
df = df.drop("商水供水1", axis=1)
df = df.drop("商水供水2", axis=1)

In [71]:
df = df.drop("供水", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 67)


In [72]:
# 处理供电
def extract_elec_giving(text):
    mindian = shangdian = 0
    if "民电" in text:
        mindian = 1
    if "商电" in text:
        shangdian = 1
    
    return mindian, shangdian

In [73]:
# 检查缺失值
target_column = "供电"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['民电供电2', '商电供电2']] = df[target_column].apply(
    lambda x: pd.Series(extract_elec_giving(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

2760
Index(['ID', '城市', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '开发商',
       '物业公司', '物业办公电话', '产权描述', '供暖', '供电', '燃气费', '供热费', '停车位', '停车费用',
       'coord_x', 'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '精装修', '是否为底层',
       '是否为低层', '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '向南', '向北', '向西',
       '向东', '交易距今', '季付', '双月付', '月付', '半年付价', '年付', '整租', '合租', '是否有电梯',
       '租用车位', '免费使用', '民电供电1', '商电供电1', '是否有燃气', '集中供暖1', '自采暖1', '租赁时间',
       '设施情况', '房屋年份', '总户数', '总楼数', '绿化率', '容积率', '物业费', '塔楼', '板楼', '平房',
       '民水供水', '商水供水', '民电供电2', '商电供电2'],
      dtype='object')
        ID  城市         lon        lat      年份     区县     板块  环线位置  \
0  2000000   1  117.345687  40.447235  2023.0   24.0  303.0   NaN   
1  2000001  10  114.279469  24.158183  2022.0  105.0  985.0   NaN   
2  2000002   3  121.684578  32.198660  2022.0   21.0  387.0   NaN   
3  2000003   0  117.323901  40.935338  2022.0   81.0  111.0  三至四环   
4  2000004   3  121.718296  32.335562  2022.0   49.0  732

In [74]:
df['民电供电'] = df.apply(lambda row: max(row["民电供电1"], row["民电供电2"]), axis=1)
df['商电供电'] = df.apply(lambda row: max(row["商电供电1"], row["商电供电2"]), axis=1)

In [75]:
df = df.drop("民电供电1", axis=1)
df = df.drop("民电供电2", axis=1)
df = df.drop("商电供电1", axis=1)
df = df.drop("商电供电2", axis=1)

In [76]:
df = df.drop("供电", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 66)


In [77]:
# 处理供暖
def extract_warm_giving(text):
    jizhong = zi = wu = 0
    if "集中供暖" in text:
        jizhong = 1
    if "自采暖" in text:
        zi = 1
    if "无供暖" in text:
        wu = 1
    
    return jizhong, zi, wu

In [78]:
# 检查缺失值
target_column = "供暖"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 应用提取函数并创建新列
df[['集中供暖2', '自采暖2', '无供暖']] = df[target_column].apply(
    lambda x: pd.Series(extract_warm_giving(x))
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

6195
Index(['ID', '城市', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '开发商',
       '物业公司', '物业办公电话', '产权描述', '供暖', '燃气费', '供热费', '停车位', '停车费用', 'coord_x',
       'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '精装修', '是否为底层', '是否为低层',
       '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '向南', '向北', '向西', '向东',
       '交易距今', '季付', '双月付', '月付', '半年付价', '年付', '整租', '合租', '是否有电梯', '租用车位',
       '免费使用', '是否有燃气', '集中供暖1', '自采暖1', '租赁时间', '设施情况', '房屋年份', '总户数', '总楼数',
       '绿化率', '容积率', '物业费', '塔楼', '板楼', '平房', '民水供水', '商水供水', '民电供电', '商电供电',
       '集中供暖2', '自采暖2', '无供暖'],
      dtype='object')
        ID  城市         lon        lat      年份     区县     板块  环线位置  \
0  2000000   1  117.345687  40.447235  2023.0   24.0  303.0   NaN   
1  2000001  10  114.279469  24.158183  2022.0  105.0  985.0   NaN   
2  2000002   3  121.684578  32.198660  2022.0   21.0  387.0   NaN   
3  2000003   0  117.323901  40.935338  2022.0   81.0  111.0  三至四环   
4  2000004   3  121.718296  32.335562  2022.0   49.0  732.0

In [79]:
df['集中供暖'] = df.apply(lambda row: max(row["集中供暖1"], row["集中供暖2"]), axis=1)
df['自采暖'] = df.apply(lambda row: max(row["自采暖1"], row["自采暖2"]), axis=1)

In [80]:
df = df.drop("集中供暖1", axis=1)
df = df.drop("集中供暖2", axis=1)
df = df.drop("自采暖1", axis=1)
df = df.drop("自采暖2", axis=1)

In [81]:
df = df.drop("供暖", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 66)


In [82]:
def extract_burn_fee(value):
    """
    参数:
    value: str - 燃气费字符串
    
    返回:
    float - 提取的燃气费数值，或-1（如果无法提取）
    """
    # 检查是否为空值
    if value == "":
        return -1
    # 转换为字符串并去除前后空格
    value_str = str(value)
    # 检查是否以"/月/m²"结尾
    if not value_str.endswith("元/m³"):
        return -1
    
    try:
        # 提取数字部分（可能包含范围符号"-"）
        # 使用正则表达式匹配数字和可能的范围
        pattern = r'([\d.-]+)元/m³'
        match = re.search(pattern, value_str)
        
        if match:
            num_part = match.group(1)
            
            # 检查是否是范围（包含"-"）
            if '-' in num_part:
                # 处理范围，取平均值
                parts = num_part.split('-')
                if len(parts) == 2:
                    start = float(parts[0])
                    end = float(parts[1])
                    return (start + end) / 2
                else:
                    return -1
            else:
                # 处理单个数值
                return float(num_part)
        else:
            return -1
    except (ValueError, AttributeError):
        return -1

In [83]:
# 检查缺失值
target_column = "燃气费"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 使用正则表达式提取数字
df[["燃气费数值"]] = df[target_column].apply(
    lambda x: pd.Series(extract_burn_fee(x), dtype='object')
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

3142
Index(['ID', '城市', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '开发商',
       '物业公司', '物业办公电话', '产权描述', '燃气费', '供热费', '停车位', '停车费用', 'coord_x',
       'coord_y', '客户反馈', '室数', '厅数', '厨房数', '卫数', '精装修', '是否为底层', '是否为低层',
       '是否为中层', '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '向南', '向北', '向西', '向东',
       '交易距今', '季付', '双月付', '月付', '半年付价', '年付', '整租', '合租', '是否有电梯', '租用车位',
       '免费使用', '是否有燃气', '租赁时间', '设施情况', '房屋年份', '总户数', '总楼数', '绿化率', '容积率',
       '物业费', '塔楼', '板楼', '平房', '民水供水', '商水供水', '民电供电', '商电供电', '无供暖', '集中供暖',
       '自采暖', '燃气费数值'],
      dtype='object')
        ID  城市         lon        lat      年份     区县     板块  环线位置  \
0  2000000   1  117.345687  40.447235  2023.0   24.0  303.0   NaN   
1  2000001  10  114.279469  24.158183  2022.0  105.0  985.0   NaN   
2  2000002   3  121.684578  32.198660  2022.0   21.0  387.0   NaN   
3  2000003   0  117.323901  40.935338  2022.0   81.0  111.0  三至四环   
4  2000004   3  121.718296  32.335562  2022.0   49.0  732.0   NaN   

     

In [84]:
df = df.drop("燃气费", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 66)


In [85]:
def extract_warm_fee(value):
    """
    参数:
    value: str - 燃气费字符串
    
    返回:
    float - 提取的燃气费数值，或-1（如果无法提取）
    """
    # 检查是否为空值
    if value == "":
        return -1
    # 转换为字符串并去除前后空格
    value_str = str(value)
    # 检查是否以"/月/m²"结尾
    if not value_str.endswith("元/㎡"):
        return -1
    
    try:
        # 提取数字部分（可能包含范围符号"-"）
        # 使用正则表达式匹配数字和可能的范围
        pattern = r'([\d.-]+)元/㎡'
        match = re.search(pattern, value_str)
        
        if match:
            num_part = match.group(1)
            
            # 检查是否是范围（包含"-"）
            if '-' in num_part:
                # 处理范围，取平均值
                parts = num_part.split('-')
                if len(parts) == 2:
                    start = float(parts[0])
                    end = float(parts[1])
                    return (start + end) / 2
                else:
                    return -1
            else:
                # 处理单个数值
                return float(num_part)
        else:
            return -1
    except (ValueError, AttributeError):
        return -1

In [86]:
# 检查缺失值
target_column = "供热费"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 使用正则表达式提取数字
df[["供热费数值"]] = df[target_column].apply(
    lambda x: pd.Series(extract_warm_fee(x), dtype='object')
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

7054
Index(['ID', '城市', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '开发商',
       '物业公司', '物业办公电话', '产权描述', '供热费', '停车位', '停车费用', 'coord_x', 'coord_y',
       '客户反馈', '室数', '厅数', '厨房数', '卫数', '精装修', '是否为底层', '是否为低层', '是否为中层',
       '是否为高层', '是否为顶层', '总楼层', '建筑面积值', '向南', '向北', '向西', '向东', '交易距今', '季付',
       '双月付', '月付', '半年付价', '年付', '整租', '合租', '是否有电梯', '租用车位', '免费使用', '是否有燃气',
       '租赁时间', '设施情况', '房屋年份', '总户数', '总楼数', '绿化率', '容积率', '物业费', '塔楼', '板楼',
       '平房', '民水供水', '商水供水', '民电供电', '商电供电', '无供暖', '集中供暖', '自采暖', '燃气费数值',
       '供热费数值'],
      dtype='object')
        ID  城市         lon        lat      年份     区县     板块  环线位置  \
0  2000000   1  117.345687  40.447235  2023.0   24.0  303.0   NaN   
1  2000001  10  114.279469  24.158183  2022.0  105.0  985.0   NaN   
2  2000002   3  121.684578  32.198660  2022.0   21.0  387.0   NaN   
3  2000003   0  117.323901  40.935338  2022.0   81.0  111.0  三至四环   
4  2000004   3  121.718296  32.335562  2022.0   49.0  732.0   NaN   

   

In [87]:
df = df.drop("供热费", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 66)


In [88]:
def extract_park(value):
    """
    参数:
    value: str - 停车位字符串
    
    返回:
    float - 提取的停车位数值，或-1（如果无法提取）
    """
    # 检查是否为空值
    if value == '':
        return -1.0
    else:
        return float(value)

In [89]:
# 检查缺失值
target_column = "停车位"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 使用正则表达式提取数字
df[["停车位数量"]] = df[target_column].apply(
    lambda x: pd.Series(extract_park(x), dtype='object')
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

3115
Index(['ID', '城市', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '开发商',
       '物业公司', '物业办公电话', '产权描述', '停车位', '停车费用', 'coord_x', 'coord_y', '客户反馈',
       '室数', '厅数', '厨房数', '卫数', '精装修', '是否为底层', '是否为低层', '是否为中层', '是否为高层',
       '是否为顶层', '总楼层', '建筑面积值', '向南', '向北', '向西', '向东', '交易距今', '季付', '双月付',
       '月付', '半年付价', '年付', '整租', '合租', '是否有电梯', '租用车位', '免费使用', '是否有燃气',
       '租赁时间', '设施情况', '房屋年份', '总户数', '总楼数', '绿化率', '容积率', '物业费', '塔楼', '板楼',
       '平房', '民水供水', '商水供水', '民电供电', '商电供电', '无供暖', '集中供暖', '自采暖', '燃气费数值',
       '供热费数值', '停车位数量'],
      dtype='object')
        ID  城市         lon        lat      年份     区县     板块  环线位置  \
0  2000000   1  117.345687  40.447235  2023.0   24.0  303.0   NaN   
1  2000001  10  114.279469  24.158183  2022.0  105.0  985.0   NaN   
2  2000002   3  121.684578  32.198660  2022.0   21.0  387.0   NaN   
3  2000003   0  117.323901  40.935338  2022.0   81.0  111.0  三至四环   
4  2000004   3  121.718296  32.335562  2022.0   49.0  732.0   NaN   

 

In [90]:
df = df.drop("停车位", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 66)


In [91]:
def extract_parking_fee(value):
    """
    参数:
    value: str - 停车费字符串
    
    返回:
    float - 提取的停车费数值，或-1.0（如果无法提取）
    """
    # 检查是否为空值
    if value == '':
        return -1.0
    
    # 转换为字符串并去除前后空格
    value_str = str(value).strip()
    
    # 检查是否只包含数字（整数或小数）
    # 使用正则表达式匹配纯数字（可包含小数点）
    pattern = r'^-?\d*\.?\d+$'
    if re.match(pattern, value_str):
        try:
            return float(value_str)
        except ValueError:
            return -1.0
    else:
        return -1.0

In [92]:
# 检查缺失值
target_column = "停车费用"
print(df[target_column].isnull().sum())
# 填充缺失值为空字符串
df[target_column] = df[target_column].fillna('')
# 使用正则表达式提取数字
df[["停车费"]] = df[target_column].apply(
    lambda x: pd.Series(extract_parking_fee(x), dtype='object')
)

print(df.columns)
print(df.head())
print(f"数据维度：{df.shape}")

3249
Index(['ID', '城市', 'lon', 'lat', '年份', '区县', '板块', '环线位置', '物业类别', '开发商',
       '物业公司', '物业办公电话', '产权描述', '停车费用', 'coord_x', 'coord_y', '客户反馈', '室数',
       '厅数', '厨房数', '卫数', '精装修', '是否为底层', '是否为低层', '是否为中层', '是否为高层', '是否为顶层',
       '总楼层', '建筑面积值', '向南', '向北', '向西', '向东', '交易距今', '季付', '双月付', '月付',
       '半年付价', '年付', '整租', '合租', '是否有电梯', '租用车位', '免费使用', '是否有燃气', '租赁时间',
       '设施情况', '房屋年份', '总户数', '总楼数', '绿化率', '容积率', '物业费', '塔楼', '板楼', '平房',
       '民水供水', '商水供水', '民电供电', '商电供电', '无供暖', '集中供暖', '自采暖', '燃气费数值', '供热费数值',
       '停车位数量', '停车费'],
      dtype='object')
        ID  城市         lon        lat      年份     区县     板块  环线位置  \
0  2000000   1  117.345687  40.447235  2023.0   24.0  303.0   NaN   
1  2000001  10  114.279469  24.158183  2022.0  105.0  985.0   NaN   
2  2000002   3  121.684578  32.198660  2022.0   21.0  387.0   NaN   
3  2000003   0  117.323901  40.935338  2022.0   81.0  111.0  三至四环   
4  2000004   3  121.718296  32.335562  2022.0   49.0  732.0   NaN   

 

In [93]:
df = df.drop("停车费用", axis=1)
print(f"数据维度：{df.shape}")

数据维度：(9773, 66)


In [94]:
df = df.drop("物业类别", axis=1)
df = df.drop("开发商", axis=1)
df = df.drop("物业公司", axis=1)
df = df.drop("物业办公电话", axis=1)
df = df.drop("产权描述", axis=1)

In [95]:
df = df.drop("环线位置", axis=1)

In [96]:
df.to_csv(f"结构化数据_{data_batch}_{data_target}.csv", index=False)